In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision
import torch.optim as optim


In [ ]:
train_dataset = torchvision.datasets.MNIST(root = './data', train = True, transform = torchvision.transforms.ToTensor(), download = True)
test_dataset = torchvision.datasets.MNIST(root = './data', train = False, transform = torchvision.transforms.ToTensor(), download = True)

In [ ]:
train_loader = DataLoader(dataset = train_dataset, batch_size = 32, shuffle = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = 32, shuffle = False)

In [ ]:
train_dataset

In [ ]:
train_images = torch.stack([image for image, label in train_dataset])  # Shape: [60000, 1, 28, 28]
train_labels = torch.tensor([label for image, label in train_dataset]) # Shape: [60000]

test_images = torch.stack([image for image, label in test_dataset])    # Shape: [10000, 1, 28, 28]
test_labels = torch.tensor([label for image, label in test_dataset])   # Shape: [10000]

In [ ]:
train_images.shape

torch.Size([60000, 1, 28, 28])

In [ ]:
train_labels.shape

torch.Size([60000])

In [ ]:
test_images.shape

torch.Size([10000, 1, 28, 28])

In [ ]:
mean_train = train_images.view(60000, -1).mean(dim = 1, keepdim = True)
std_train  = train_images.view(60000, -1).std(dim = 1, keepdim = True)
mean_test = test_images.view(10000, -1).mean(dim = 1, keepdim = True)
std_test  = test_images.view(10000, -1).std(dim = 1, keepdim = True)

In [ ]:
train_images_normalized = (train_images.view(60000,-1) - mean_train)/std_train
test_images_normalized = (test_images.view(10000,-1) - mean_test)/std_test

In [ ]:
train_images_normalized = train_images_normalized.view(60000,28,28)
test_images_normalized = test_images_normalized.view(10000,28,28)

In [ ]:
filter_height = torch.tensor([1, 3, 5, 7])
filter_width = torch.tensor([1, 3, 5, 7])
num_of_filters = torch.tensor([4, 8, 16, 32])
search_space = torch.stack([filter_height,filter_width,num_of_filters], dim = 0)

In [ ]:
filter_height.shape

torch.Size([4])

In [ ]:
search_space

tensor([[ 1,  3,  5,  7],
        [ 1,  3,  5,  7],
        [ 4,  8, 16, 32]])

In [ ]:
batch_size_RNN = 1
input_size = 1
seq_len = 12
num_layers = 3
num_attribute = 4
num_layers_attributes = 3

In [ ]:
class ControllerRNN(nn.Module):
  def __init__(self, input_size = 1, hidden_size = 32):
    super().__init__()
    self.hidden_size = hidden_size
    self.rnn = nn.RNNCell(input_size, hidden_size)
    self.logits = nn.Linear(hidden_size, 4)
    self.W_prev = torch.randn(hidden_size, hidden_size)
    self.W_curr = torch.randn(hidden_size, hidden_size)
    self.v = torch.randn(hidden_size)


  def forward(self):
    x_t = torch.zeros(batch_size_RNN, input_size)
    h_t = torch.ones(batch_size_RNN, self.hidden_size)
    outputs = []
    attention = []
    skip_probs = []
    max_probs = []
    for i in range(num_layers):
      for j in range(num_attribute):
        if j == 3:
          h_t = self.rnn(x_t,h_t)
          attention.append(h_t)

          if len(attention) > 1:
            scores = []

            for prev_h in attention[:-1]:  # all previous anchors
                z = torch.tanh(prev_h @ self.W_prev + h_t @ self.W_curr)
                scores.append(torch.sigmoid(torch.matmul(z, self.v)).item())

            skip_probs.append(scores)

        else:
          h_t = self.rnn(x_t, h_t)
          logits = self.logits(h_t)
          prob = F.softmax(logits, dim = 1)
          predicted_index = torch.argmax(prob, dim=1)
          x_t = search_space[j][predicted_index].unsqueeze(1).float()
          max_probs.append(torch.max(prob))
          outputs.append(x_t.item())
    return outputs, skip_probs, max_probs

In [ ]:
rnn = ControllerRNN()outputs,skip_probs, max_probs = rnn.forward()

In [ ]:
max_probs[0]

tensor(0.3250, grad_fn=<MaxBackward1>)

In [ ]:
skip_probs

[[0.9959191679954529], [0.9959294199943542, 0.9964578747749329]]

In [ ]:
outputs = [int(x) for x in outputs]

In [ ]:
outputs

[3, 7, 32, 7, 7, 32, 7, 7, 32]

In [ ]:
class ChildNetwork(nn.Module):
  def __init__(self, input_channels = 1,batch_size = 1, outputs = [] ,skip_prob = [] ):
    super().__init__()
    self.in_channel = input_channels
    self.convs = nn.ModuleList()
    self.linear = None
    self.batch_size = batch_size

    for i in range(num_layers):
        attributes = outputs[i*num_layers_attributes:(i+1)*num_layers_attributes]
        out_channels = attributes[2]
        kernel_height = attributes[0]
        kernel_width = attributes[1]
        conv = nn.Conv2d(
            in_channels=self.in_channel,
            out_channels=out_channels,
            kernel_size=(kernel_height, kernel_width)
        )
        self.convs.append(conv)
        if i < num_layers - 1:
          prev_probs = skip_probs[i]
          index = prev_probs.index(max(prev_probs))
          self.in_channel = outputs[index*num_layers_attributes + (num_layers_attributes-1)]

  def forward(self, x):
    layer_outputs = []
    non_used_layers = [1] + [0] * (num_layers - 1)
    non_used_layers_outputs = []
    for i, conv in enumerate(self.convs):
        if i >0:
          prev_probs = skip_probs[i-1]
          index = prev_probs.index(max(prev_probs))
          x = conv(layer_outputs[index])
          non_used_layers[index] = 1
        else:
          x = F.relu(conv(x))
        layer_outputs.append(x)
    for i in range(len(non_used_layers)):
      if non_used_layers[i] == 0:
        non_used_layers_outputs.append(layer_outputs[i])

    max_h = max([x.shape[2] for x in non_used_layers_outputs])
    max_w = max([x.shape[3] for x in non_used_layers_outputs])

    padded_outputs = []
    for out in non_used_layers_outputs:
        _, _, h, w = out.shape
        pad_h = max_h - h
        pad_w = max_w - w
        # Pad (left, right, top, bottom)
        padded = F.pad(out, (0, pad_w, 0, pad_h))
        padded_outputs.append(padded)

    combined = torch.cat(padded_outputs, dim=1)
    flattened = combined.view(x.shape[0], -1)
    if self.linear is None:
        input_features = flattened.shape[1]
        self.linear = nn.Linear(input_features, 10).to(flattened.device)
    output = self.linear(flattened)
    return output


In [ ]:
batch_size = 32

In [ ]:
train_images_normalized = train_images_normalized.unsqueeze(1)

In [ ]:
test_images_normalized = test_images_normalized.unsqueeze(1)

In [ ]:
def train_test(model, X_train, y_train, X_test, y_test, optimizer, criterion, batch_size = 32, epochs = 1000):
  model.train()
  for i in range(epochs):
    total_loss = 0.0
    idx = torch.randint(low = 0, high = X_train.shape[0]-batch_size, size = (batch_size,))
    X_train_batch = X_train[idx]
    y_train_batch = y_train[idx]
    optimizer.zero_grad()
    output = model(X_train_batch)
    loss = criterion(output, y_train_batch)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()


  model.eval()
  with torch.no_grad():
    output = model(X_test)
    _, predicted = torch.max(output, 1)
    correct = (predicted == y_test).sum().item()
    accuracy = correct / y_test.size(0)

  return accuracy

In [ ]:
##Train loop
controller_optimizer = optim.Adam(rnn.parameters(), lr = 0.01)
num_epochs = 10
for i in range(num_epochs):
  loss = 0
  for p in max_probs:
    log_p = torch.log(p)
    loss -= log_p * reward  # negative for gradient ascent

loss.backward()
controller_optimizer.step()


In [ ]:
rnn = ControllerRNN()

In [ ]:
num_epochs = 10
final_batch_size = 20
controller_optimizer = optim.Adam(rnn.parameters(), lr=0.01)

best_accuracy = 0.0
best_architecture = None
best_skip_prob = None

for epoch in range(num_epochs):
    controller_optimizer.zero_grad()
    total_loss = 0.0

    for j in range(final_batch_size):
        outputs, skip_probs, max_probs = rnn.forward()
        outputs = [int(x) for x in outputs]

        # Build and train the child model
        CN = ChildNetwork(outputs=outputs, skip_prob=skip_probs)
        optimizer = optim.Adam(CN.parameters(), lr=0.01)
        criterion = nn.CrossEntropyLoss()

        accuracy = train_test(
            CN, train_images_normalized, train_labels,
            test_images_normalized, test_labels,
            optimizer=optimizer, criterion=criterion,
            batch_size=32, epochs=100
        )

        # Save best architecture based on accuracy
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_architecture = outputs
            best_skip_prob = skip_probs  # if required

        # REINFORCE loss
        reward = torch.tensor(accuracy, dtype=torch.float32)
        loss = 0
        for p in max_probs:
            log_p = torch.log(p + 1e-8)
            loss -= log_p * reward
        total_loss += loss

    total_loss.backward()
    controller_optimizer.step()
    print(f'Epoch {epoch}, Loss: {total_loss.item():.4f}, Best Acc: {best_accuracy:.4f}')


Epoch 0, Loss: 177.1315, Best Acc: 0.8437
Epoch 1, Loss: 147.3927, Best Acc: 0.9141
Epoch 2, Loss: 90.9721, Best Acc: 0.9141
Epoch 3, Loss: 53.5206, Best Acc: 0.9141
Epoch 4, Loss: 31.8494, Best Acc: 0.9141
Epoch 5, Loss: 19.8428, Best Acc: 0.9141
Epoch 6, Loss: 12.4181, Best Acc: 0.9141
Epoch 7, Loss: 7.7775, Best Acc: 0.9141
Epoch 8, Loss: 4.9475, Best Acc: 0.9173
Epoch 9, Loss: 3.2429, Best Acc: 0.9173


In [ ]:
# Evaluate or reuse the best architecture found
print("Best architecture:", best_architecture)

# Build and train final child model more thoroughly
CN = ChildNetwork(outputs=best_architecture, skip_prob=best_skip_prob)
optimizer = optim.Adam(CN.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

final_accuracy = train_test(
    CN, train_images_normalized, train_labels,
    test_images_normalized, test_labels,
    optimizer=optimizer, criterion=criterion,
    batch_size=32, epochs=300  # train longer this time
)

print(f"Final accuracy of best architecture: {final_accuracy:.4f}")


Best architecture: [5, 5, 16, 5, 5, 16, 5, 5, 16]
Final accuracy of best architecture: 0.9202


In [ ]:
outputs

[3, 1, 16, 1, 5, 4, 1, 5, 4]

In [ ]:
rnn.eval()
num_samples = 30
sampled_architectures = []

for _ in range(num_samples):
    outputs, skip_probs, max_probs = rnn.forward()
    arch = {
        'outputs': [int(x) for x in outputs],
        'skip_probs': skip_probs,
        'max_probs': max_probs
    }
    sampled_architectures.append(arch)


In [ ]:
accuracies = []

for arch in sampled_architectures:
    CN = ChildNetwork(outputs=arch['outputs'], skip_prob=arch['skip_probs'])
    optimizer = optim.Adam(CN.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()

    acc = train_test(
        CN, train_images_normalized, train_labels,
        test_images_normalized, test_labels,
        optimizer=optimizer, criterion=criterion,
        batch_size=32, epochs=20  # Shorter training for speed
    )
    accuracies.append(acc)


In [ ]:
best_index = torch.tensor(accuracies).argmax().item()
best_architecture = sampled_architectures[best_index]

print("Best Accuracy:", accuracies[best_index])
print("Best Outputs:", best_architecture['outputs'])


Best Accuracy: 0.7993
Best Outputs: [5, 5, 16, 5, 5, 16, 5, 5, 16]


In [ ]:
CN_best = ChildNetwork(outputs=best_architecture['outputs'], skip_prob=best_architecture['skip_probs'])
optimizer = optim.Adam(CN_best.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

final_accuracy = train_test(
    CN_best, train_images_normalized, train_labels,
    test_images_normalized, test_labels,
    optimizer=optimizer, criterion=criterion,
    batch_size=32, epochs=300  # Train longer
)

print("Final accuracy of best architecture:", final_accuracy)

Final accuracy of best architecture: 0.9244
